In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train=pd.read_csv("/kaggle/input/sentiment-prediction-on-movie-reviews/train.csv")
test=pd.read_csv("/kaggle/input/sentiment-prediction-on-movie-reviews/test.csv")
movies=pd.read_csv("/kaggle/input/sentiment-prediction-on-movie-reviews/movies.csv")

movies.info()

In [ ]:
train.shape, movies.shape

# **Exploratory Data Analysis**

In [ ]:
movies.describe()

In [ ]:
train.describe()

In [ ]:
movies.info()

**Null Values in movies dataset**

In [ ]:
print("NULL VALUES")
for col in movies.columns:
    nas= 100* movies[col].isna().sum()/143258 
    print(f"{col}: {nas}%")

In [ ]:
sns.heatmap(movies.isnull(), yticklabels=False, cbar= False, cmap= 'viridis')

**Null values in train dataset**

In [ ]:
sns.heatmap(train.isnull(), yticklabels=False, cbar= False, cmap= 'viridis')

In [ ]:
# Removing features with more than 48% Null values from 'movies'
movies.drop(["rating","ratingContents","releaseDateTheaters","boxOffice","distributor","soundType","title","audienceScore"],axis=1,inplace=True)
movies.head(5)

In [ ]:
movies.shape, train.shape

In [ ]:
train.head()

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x= 'isFrequentReviewer', data= train)

In [ ]:
train['sentiment'].value_counts()

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x= 'isFrequentReviewer',hue= 'sentiment', data= train)

In [ ]:
sns.distplot(movies['runtimeMinutes'].dropna(), bins=40)

In [ ]:
for col in movies.columns:
    nas= 100* movies[col].isna().sum()/143258 
    print(f"{col} {nas}%")

In [ ]:
movies.drop_duplicates(subset= ["movieid",], inplace=True)

In [ ]:
movies.shape

In [ ]:
train.head()

In [ ]:
train= train.merge(movies,on= "movieid", how="left")
test= test.merge(movies,on= "movieid",  how="left")

In [ ]:
train.head()

In [ ]:
train.shape, test.shape

In [ ]:
for col in train.columns:
    nas= 100* train[col].isna().sum()/162758 
    print(f"{col} {nas}%")

In [ ]:
train["reviewText"].fillna(train["reviewText"].mode()[0],inplace= True)
test["reviewText"].fillna(test["reviewText"].mode()[0],inplace= True)

In [ ]:
train['reviewText'].isna().sum(), test['reviewText'].isna().sum()

In [ ]:
# changing 'releaseDateStreaming' to Integer type (keeping it as a count from the oldest date)
train['releaseDateStreaming'] = pd.to_datetime(train['releaseDateStreaming'])
start= min(train["releaseDateStreaming"])
list= []
for val in train["releaseDateStreaming"]:
    diff= (val- start).total_seconds()//216000
    list.append(diff)
train["releaseDateStreaming"]= pd.Series(list)

# encoding isFrequentReviewer as 0 and 1
train['isFrequentReviewer'].replace({True:1,False:0}, inplace=True)

# making same tranformations to test data
test['releaseDateStreaming'] = pd.to_datetime(test['releaseDateStreaming'])
start= min(test["releaseDateStreaming"])
list= []
for val in test["releaseDateStreaming"]:
    diff= (val- start).total_seconds()//216000
    list.append(diff)
test["releaseDateStreaming"]= pd.Series(list)
test.rename(columns={'isTopCritic': 'isFrequentReviewer'}, inplace=True)
test['isFrequentReviewer'].replace({True:1,False:0}, inplace=True)

In [ ]:
# The genre feature had multiple genres of movie written in the same string, so we split them
# Example : "Comedy, Animation, Adventure, Fantasy"
# Also for rows whitespace is given between the genres and other places it is not.

genre= set()
lis= []
for cat in train["genre"]:
    dict= {}
    if type(cat)!= str:
        lis.append(dict)
        continue
    for i in cat.split(","):
        genre.add(i.strip())
        dict[i.strip()]= 1
    lis.append(dict)

list_test= []
for cat in test["genre"]:
    dict= {}
    if type(cat)!= str:
        list_test.append(dict)
        continue
    for i in cat.split(","):
        genre.add(i.strip())
        dict[i.strip()]= 1
    list_test.append(dict)
genre= [i for i in genre]
genre_df= pd.DataFrame(lis,columns=genre)
genre_df.fillna(0, inplace= True)

genre_df_test= pd.DataFrame(list_test, columns=genre)
genre_df_test.fillna(0, inplace= True)

In [ ]:
genre_df.shape, genre_df_test.shape

In [ ]:
train.drop("genre", axis= 1, inplace= True)
train= pd.concat([train, genre_df], axis=1)

test.drop("genre", axis= 1, inplace= True)
test= pd.concat([test, genre_df_test], axis=1)

In [ ]:
train.shape, test.shape

In [ ]:
train['reviewText'][3]

In [ ]:
# Removing all the special charaters from the ReviewText feature

def remove_spec(txt):
    x=''
    for i in txt:
        if i.isalnum():
            x = x+i
        else:
            x= x+' '
    return x
train['reviewText'] = train['reviewText'].apply(remove_spec)
test['reviewText'] = test['reviewText'].apply(remove_spec)

In [ ]:
train['reviewText'][3]

In [ ]:
train.shape

In [ ]:
# Separating the Label from the train data, and encoding it as 0 and 1

Y= train["sentiment"]
Y.replace({"POSITIVE":1,"NEGATIVE":0}, inplace=True)
train.drop("sentiment", axis= 1, inplace= True)

**Vectorization of the ReviewText column**

In [ ]:
# Using TfidfVectorizer with stop words

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tfidf= TfidfVectorizer(stop_words='english')
X_rt= tfidf.fit_transform(train["reviewText"])
X_rt_test= tfidf.transform(test["reviewText"])
type(X_rt)

In [ ]:
# We concat these sparse matrices to train and test data after other preprocessing steps
X_rt, X_rt_test

In [ ]:
train.drop("reviewText", axis= 1, inplace= True)
test.drop("reviewText", axis= 1, inplace= True)

In [ ]:
train.shape, test.shape

**Imputing, Encoding, Scaling of train and test data**

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import Pipeline

In [ ]:
number_pipeline=Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),
                                 ('scaler1', StandardScaler()),
                                ])
categorical_pipeline=Pipeline(steps=[('imputer',SimpleImputer(strategy='most_frequent')),
                                     ('OneHotEncoding', OneHotEncoder(handle_unknown='ignore'))
                                    ])

In [ ]:
cat_cols = train.select_dtypes(include = 'object').columns
num_cols = train.select_dtypes(include = 'float64').columns

clt = ColumnTransformer([('number_pipeline',number_pipeline,num_cols ),
                         ('categorical_pipeline',categorical_pipeline, cat_cols),
                         ])
train = clt.fit_transform(train)
test = clt.transform(test)

In [ ]:
# Now we concat the train and test data with the transformed reviewText sparse matrix

from scipy.sparse import hstack

train= hstack((train, X_rt))
test= hstack((test, X_rt_test))

**Splitting of train data**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, make_scorer, accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils import shuffle

X_train,X_test,Y_train,Y_test = train_test_split(train,Y,test_size = 0.15,random_state = 42, shuffle= True)

In [ ]:
X_train

# **SGDClassifier**

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
model= SGDClassifier()
gscv_sgd= GridSearchCV(estimator=model,
                       param_grid ={"loss":["hinge", "log", "modified_huber"], 
                                    "alpha":[0.001,0.01]},
                       scoring={"accuracy_score":make_scorer(accuracy_score)},
                       refit="accuracy_score",
                       cv=3,
                       n_jobs=-1)

In [ ]:
X_train.shape, Y_train.shape

In [ ]:
gscv_sgd.fit(X_train,Y_train)
pd.DataFrame(gscv_sgd.cv_results_)

In [ ]:
gscv_sgd.best_score_, gscv_sgd.best_params_
# 0.783170941994007, {'alpha': 0.001, 'loss': 'modified_huber'}

In [ ]:
Y_pred= gscv_sgd.predict(X_test)
print(classification_report(Y_pred, Y_test))

In [ ]:
cm= confusion_matrix(Y_test, Y_pred)
cm_display= ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = [0,1])
cm_display.plot()
plt.show()

# **Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
model= LogisticRegression(warm_start= False, max_iter= 1000)
model.get_params()

In [ ]:
gscv_logr= GridSearchCV(estimator=model,
                       param_grid ={ "solver":["saga","sag", "newton-cg"], "C": [0.01,0.001,0.0001]},
                       scoring={"accuracy_score":make_scorer(accuracy_score)},
                       refit="accuracy_score",
                       cv=4,
                       n_jobs=-1)

In [ ]:
gscv_logr.fit(X_train,Y_train)
pd.DataFrame(gscv_logr.cv_results_)

In [ ]:
gscv_logr.best_params_, gscv_logr.best_score_
# {'C': 0.01, 'solver': 'sag'}, 0.7067744173943213

In [ ]:
Y_pred= gscv_logr.predict(X_test)
print(classification_report(Y_pred, Y_test))

In [ ]:
cm= confusion_matrix(Y_test, Y_pred)
cm_display= ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = [0,1])
cm_display.plot()
plt.show()

# **Linear SVC**

In [ ]:
from sklearn.svm import LinearSVC
model= LinearSVC()
model.get_params()

In [ ]:
gscv_svc= GridSearchCV(estimator=model,
                       param_grid ={'C': [0.01,0.1, 1], },
                       scoring={"accuracy_score":make_scorer(accuracy_score)},
                       refit="accuracy_score",
                       cv=4,
                       n_jobs=-1)

In [ ]:
gscv_svc.fit(X_train,Y_train)
pd.DataFrame(gscv_svc.cv_results_)

In [ ]:
#  Linear SVC giving the best score of 0.82044
gscv_svc.best_params_, gscv_svc.best_score_

In [ ]:
Y_pred= gscv_svc.predict(X_test)
print(classification_report(Y_pred, Y_test))

In [ ]:
cm= confusion_matrix(Y_test, Y_pred)
cm_display= ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = [0,1])
cm_display.plot()
plt.show()

# **Bernoulli Naive Bayes**

In [ ]:
from sklearn.naive_bayes import BernoulliNB
model =BernoulliNB()
model.get_params()

In [ ]:
gscv_nb= GridSearchCV(estimator=model,
                       param_grid={"alpha":[1,0.1,0.01]},
                       scoring={"accuracy_score":make_scorer(accuracy_score)},
                       refit="accuracy_score",
                       cv=4,
                       n_jobs=-1)

In [ ]:
gscv_nb.fit(X_train,Y_train)
pd.DataFrame(gscv_nb.cv_results_)

In [ ]:
gscv_nb.best_params_, gscv_nb.best_score_
# {'alpha': 1}, 0.8035838200427919

In [ ]:
Y_pred= gscv_nb.predict(X_test)
print(classification_report(Y_pred, Y_test))

In [ ]:
cm= confusion_matrix(Y_test, Y_pred)
cm_display= ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = [0,1])
cm_display.plot()
plt.show()

# **Final prediction**

In [ ]:
# We use our best model, which is Linear SVC for prediction 

Y_pred_test = gscv_svc.predict(test)
Y_pred_test

In [ ]:
pd.Series(Y_pred_test).value_counts()

In [ ]:
n,d= test.shape
sample=pd.read_csv("/kaggle/input/sentiment-prediction-on-movie-reviews/sample.csv")

submission=pd.DataFrame(columns=['id', 'sentiment'])
submission['id']=[i for i in range(n)]
lis= []
for i in Y_pred_test:
    if i==0:
        lis.append("NEGATIVE")
    else:
        lis.append("POSITIVE")
submission["sentiment"]= pd.Series(lis)
submission.to_csv('submission.csv',index=False)
submission